<a href="https://colab.research.google.com/github/disha-ramchandani/P2P-loans/blob/main/XGBModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [5]:
df=pd.read_excel('clean_data.xlsx')

In [6]:
df.head()

,Unnamed: 0,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_to_income_ratio,previous_default,credit_history_length
0,1,21,9600,OWN,5,EDUCATION,B,1000,11.14,0,0.10,N,2
1,2,25,9600,MORTGAGE,1,MEDICAL,C,5500,12.87,1,0.57,N,3
2,3,23,65500,RENT,4,MEDICAL,C,35000,15.23,1,0.53,N,2
3,4,24,54400,RENT,8,MEDICAL,C,35000,14.27,1,0.55,Y,4
4,5,21,9900,OWN,2,VENTURE,A,2500,7.14,1,0.25,N,2


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28602 entries, 0 to 28601
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             28602 non-null  int64  
 1   person_age             28602 non-null  int64  
 2   person_income          28602 non-null  int64  
 3   person_home_ownership  28602 non-null  object 
 4   person_emp_length      28602 non-null  int64  
 5   loan_intent            28602 non-null  object 
 6   loan_grade             28602 non-null  object 
 7   loan_amnt              28602 non-null  int64  
 8   loan_int_rate          28602 non-null  float64
 9   loan_status            28602 non-null  int64  
 10  loan_to_income_ratio   28602 non-null  float64
 11  previous_default       28602 non-null  object 
 12  credit_history_length  28602 non-null  int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 2.8+ MB


In [14]:
df['person_home_ownership'].replace({'OWN':0,'RENT':1,'MORTGAGE':2,'OTHER':3},inplace=True)
df['loan_intent'].replace({'EDUCATION':0, 'MEDICAL':1, 'VENTURE':2, 'PERSONAL':3, 'HOMEIMPROVEMENT':4,'DEBTCONSOLIDATION':5},inplace=True)
df['loan_grade'].replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6},inplace=True)
df['previous_default'].replace({'Y':1,'N':0},inplace=True)


In [13]:
df['previous_default'].unique()


array(['N', 'Y'], dtype=object)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28602 entries, 0 to 28601
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             28602 non-null  int64  
 1   person_age             28602 non-null  int64  
 2   person_income          28602 non-null  int64  
 3   person_home_ownership  28602 non-null  int64  
 4   person_emp_length      28602 non-null  int64  
 5   loan_intent            28602 non-null  int64  
 6   loan_grade             28602 non-null  int64  
 7   loan_amnt              28602 non-null  int64  
 8   loan_int_rate          28602 non-null  float64
 9   loan_status            28602 non-null  int64  
 10  loan_to_income_ratio   28602 non-null  float64
 11  previous_default       28602 non-null  int64  
 12  credit_history_length  28602 non-null  int64  
dtypes: float64(2), int64(11)
memory usage: 2.8 MB


In [16]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_testcek, y_train, y_testcek = train_test_split(df[['person_age','person_income','person_home_ownership','person_emp_length','loan_intent','loan_amnt','loan_int_rate','loan_status','loan_to_income_ratio','previous_default','credit_history_length']],df['loan_grade'],test_size=0.25)

X_test = df[['person_age','person_income','person_home_ownership','person_emp_length','loan_intent','loan_amnt','loan_int_rate','loan_status','loan_to_income_ratio','previous_default','credit_history_length']] 

In [21]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [22]:
xgb = XGBClassifier(learning_rate =0.1,
                    n_estimators=1000,
                    max_depth=5,
                    min_child_weight=2,
                    gamma=0,
                    subsample=0.5,
                    colsample_bytree=0.6,
                    scale_pos_weight=1)
model = xgb.fit(X_train, y_train)
fits = xgb.predict(X_train)
predscek = xgb.predict(X_testcek)
acc_xgbfits = (fits == y_train).sum().astype(float) / len(fits)*100
acc_xgbcek = (predscek == y_testcek).sum().astype(float) / len(predscek)*100
print("XGBoost's prediction accuracy for training data is: %3.2f" % (acc_xgbfits))
print("XGBoost's prediction accuracy for testing data in sample is: %3.2f" % (acc_xgbcek))
preds = xgb.predict(X_test)

XGBoost's prediction accuracy for training data is: 100.00
XGBoost's prediction accuracy for testing data in sample is: 97.25


In [38]:
submission=pd.DataFrame({'age':df['person_age'],'loan_grade':preds})

In [39]:
submission['loan_grade'].replace({0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G'}, inplace=True)

In [40]:
submission

,age,loan_grade
0,21,B
1,25,C
2,23,C
3,24,C
4,21,A
...,...,...
28597,52,D
28598,52,B
28599,57,C
28600,54,A
